<a href="https://colab.research.google.com/github/lugastyan98/portofolio/blob/main/rekomendation_system_versi_menggunakan_data_angka_angka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# library

In [ ]:
!pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PINA dataset/semua_versi_4___edwin_2023-05-10T04_25_24.475027Z.csv')

In [ ]:
df_user = pd.read_csv('/content/drive/MyDrive/PINA dataset/user_portfolio.csv')

In [ ]:
df_user.head()

,clientportfolioclientid,clientportfoliostockid,clientportfoliobalance,clientportfoliodate,date_rank_per_user
0,0000003,FIMP,19000000.0,2023-05-05,1
1,01OLK,BOLT,100.0,2023-05-05,1
2,3,CASH,47051800.0,2022-03-28,1
3,ADANGWAH107,TLKM,100.0,2023-05-05,1
4,ADIN99,TRIM,100.0,2023-05-05,1


In [ ]:
df_grouped = pd.DataFrame(df_user.groupby('clientportfolioclientid')['clientportfoliostockid'].agg(list))

In [ ]:
df_grouped = df_grouped.reset_index()

In [ ]:
df_grouped.head()

,clientportfolioclientid,clientportfoliostockid
0,0000003,[FIMP]
1,01OLK,[BOLT]
2,3,[CASH]
3,ADANGWAH107,[TLKM]
4,ADIN99,[TRIM]


In [ ]:
df_grouped.columns  = ['id','portofolio']

In [ ]:
df_grouped.head()

,id,portofolio
0,0000003,[FIMP]
1,01OLK,[BOLT]
2,3,[CASH]
3,ADANGWAH107,[TLKM]
4,ADIN99,[TRIM]


In [ ]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          555 non-null    object
 1   portofolio  555 non-null    object
dtypes: object(2)
memory usage: 8.8+ KB


In [ ]:
print(df_grouped['id'].nunique())

555


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             865 non-null    object 
 5   business_product         669 non-null    object 
 6   total_asset_growth       853 non-null    float64
 7   price_sales_ratio        845 non-null    float64
 8   sales_growth             834 non-null    float64
 9   quick_ratio              849 non-null    float64
 10  cost_of_goods_sold       853 non-null    float64
 11  cash_flow_per_share      850 non-null    float64
 12  net_income_growth        839 non-null    float64
 13  operating_profit         853 non-null    float64
 14  income_before_tax        8

# Missing Value

Missing value akan diisi dengan rata-rata.
Tidak dihapus karena data _id itu merupakan data semua saham di PINA


In [ ]:
df.isnull().sum() #cek yang NaN

_id                          0
sector                       0
idx                          0
thematic                     0
general_info                18
business_product           214
total_asset_growth          30
price_sales_ratio           38
sales_growth                49
quick_ratio                 34
cost_of_goods_sold          30
cash_flow_per_share         33
net_income_growth           44
operating_profit            30
income_before_tax           30
total_assets                30
total_equity                30
total_liabilities           30
debt_to_equity_ratio        30
cash_equivalent             30
total_sales                 30
operating_profit_margin     35
gross_profit_margin         35
net_profit_margin           35
net_income                  30
market_cap                  22
dailyReturn                 22
monthlyReturn               22
annualReturn                22
threeYearReturn             22
fiveYearReturn              22
tenYearReturn               94
listedSh

In [ ]:
count = df['_id'].nunique()
print(count)

883


In [ ]:
df.fillna(df.mean(), inplace=True) #mengisi dengan nilai rata-rata

<ipython-input-136-9598921fdd30>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True) #mengisi dengan nilai rata-rata


In [ ]:
df.isnull().sum() #cek yang NaN

_id                          0
sector                       0
idx                          0
thematic                     0
general_info                18
business_product           214
total_asset_growth           0
price_sales_ratio            0
sales_growth                 0
quick_ratio                  0
cost_of_goods_sold           0
cash_flow_per_share          0
net_income_growth            0
operating_profit             0
income_before_tax            0
total_assets                 0
total_equity                 0
total_liabilities            0
debt_to_equity_ratio         0
cash_equivalent              0
total_sales                  0
operating_profit_margin      0
gross_profit_margin          0
net_profit_margin            0
net_income                   0
market_cap                   0
dailyReturn                  0
monthlyReturn                0
annualReturn                 0
threeYearReturn              0
fiveYearReturn               0
tenYearReturn                0
listedSh

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             865 non-null    object 
 5   business_product         669 non-null    object 
 6   total_asset_growth       883 non-null    float64
 7   price_sales_ratio        883 non-null    float64
 8   sales_growth             883 non-null    float64
 9   quick_ratio              883 non-null    float64
 10  cost_of_goods_sold       883 non-null    float64
 11  cash_flow_per_share      883 non-null    float64
 12  net_income_growth        883 non-null    float64
 13  operating_profit         883 non-null    float64
 14  income_before_tax        8

# Aku mau hapus kolum tidak dibutuhkan ( kolum berupa string)

In [ ]:
#menghapus kolum yang string
columns_to_drop = ['sector','idx','thematic','general_info','business_product','description','underwriter','shareholders','industry']
df = df.drop(columns=columns_to_drop)


In [ ]:
df.set_index('_id', inplace = True)
df.head()

,total_asset_growth,price_sales_ratio,sales_growth,quick_ratio,cost_of_goods_sold,cash_flow_per_share,net_income_growth,operating_profit,income_before_tax,total_assets,...,lowTradedPrice,highTradedPrice,bidPrice,offerPrice,book_value_per_share,eps,pbv,roa,roe,per
_id,,,,,,,,,,,,,,,,,,,,,
AALI,0.314978,0.707587,-0.102518,2.005423,-1.800647e+13,953.607381,-0.124157,2.360413e+12,2.429178e+12,2.924934e+13,...,7625.0,7650.0,7650.0,7675.0,11556.791102,432.84,0.65260,0.059031,0.077624,10.22214
ABBA,12.219118,3.491533,0.008439,1.051620,-9.834737e+10,-24.442054,-0.011553,-3.219792e+10,-2.743303e+10,3.888845e+11,...,103.0,107.0,104.0,105.0,7.474376,-9.49,-50.47942,-0.056624,-0.748516,-10.88109
ABDA,0.642063,5.343024,0.001388,80.622178,-2.570245e+11,-58.453396,-0.420802,1.186289e+11,1.187926e+11,2.472106e+12,...,0.0,0.0,6000.0,6450.0,2425.051349,237.60,2.66314,0.036869,0.060541,22.02523
ABMM,2.210543,0.397122,0.559536,1.055253,-1.452951e+13,2861.056482,1.010521,6.225255e+12,6.841950e+12,3.118794e+13,...,3540.0,3750.0,3690.0,3700.0,3528.385395,580.73,0.88301,0.136141,0.437086,4.08047
ACES,0.221650,1.257822,0.033536,3.810510,-3.494851e+12,36.083874,-0.056846,8.760908e+11,8.208312e+11,7.249255e+12,...,555.0,560.0,550.0,555.0,346.005168,30.69,1.56185,0.091643,0.111956,24.38614


In [ ]:
df.head()

,total_asset_growth,price_sales_ratio,sales_growth,quick_ratio,cost_of_goods_sold,cash_flow_per_share,net_income_growth,operating_profit,income_before_tax,total_assets,...,lowTradedPrice,highTradedPrice,bidPrice,offerPrice,book_value_per_share,eps,pbv,roa,roe,per
_id,,,,,,,,,,,,,,,,,,,,,
AALI,0.314978,0.707587,-0.102518,2.005423,-1.800647e+13,953.607381,-0.124157,2.360413e+12,2.429178e+12,2.924934e+13,...,7625.0,7650.0,7650.0,7675.0,11556.791102,432.84,0.65260,0.059031,0.077624,10.22214
ABBA,12.219118,3.491533,0.008439,1.051620,-9.834737e+10,-24.442054,-0.011553,-3.219792e+10,-2.743303e+10,3.888845e+11,...,103.0,107.0,104.0,105.0,7.474376,-9.49,-50.47942,-0.056624,-0.748516,-10.88109
ABDA,0.642063,5.343024,0.001388,80.622178,-2.570245e+11,-58.453396,-0.420802,1.186289e+11,1.187926e+11,2.472106e+12,...,0.0,0.0,6000.0,6450.0,2425.051349,237.60,2.66314,0.036869,0.060541,22.02523
ABMM,2.210543,0.397122,0.559536,1.055253,-1.452951e+13,2861.056482,1.010521,6.225255e+12,6.841950e+12,3.118794e+13,...,3540.0,3750.0,3690.0,3700.0,3528.385395,580.73,0.88301,0.136141,0.437086,4.08047
ACES,0.221650,1.257822,0.033536,3.810510,-3.494851e+12,36.083874,-0.056846,8.760908e+11,8.208312e+11,7.249255e+12,...,555.0,560.0,550.0,555.0,346.005168,30.69,1.56185,0.091643,0.111956,24.38614


# Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_data = scaler.fit_transform(df)

In [ ]:
type(scaled_data)

numpy.ndarray

# Bagian Baru

In [ ]:
df

,total_asset_growth,price_sales_ratio,sales_growth,quick_ratio,cost_of_goods_sold,cash_flow_per_share,net_income_growth,operating_profit,income_before_tax,total_assets,...,lowTradedPrice,highTradedPrice,bidPrice,offerPrice,book_value_per_share,eps,pbv,roa,roe,per
_id,,,,,,,,,,,,,,,,,,,,,
AALI,0.314978,0.707587,-0.102518,2.005423,-1.800647e+13,953.607381,-0.124157,2.360413e+12,2.429178e+12,2.924934e+13,...,7625.0,7650.0,7650.000000,7675.0,11556.791102,4.328400e+02,0.65260,0.059031,0.077624,10.222140
ABBA,12.219118,3.491533,0.008439,1.051620,-9.834737e+10,-24.442054,-0.011553,-3.219792e+10,-2.743303e+10,3.888845e+11,...,103.0,107.0,104.000000,105.0,7.474376,-9.490000e+00,-50.47942,-0.056624,-0.748516,-10.881090
ABDA,0.642063,5.343024,0.001388,80.622178,-2.570245e+11,-58.453396,-0.420802,1.186289e+11,1.187926e+11,2.472106e+12,...,0.0,0.0,6000.000000,6450.0,2425.051349,2.376000e+02,2.66314,0.036869,0.060541,22.025230
ABMM,2.210543,0.397122,0.559536,1.055253,-1.452951e+13,2861.056482,1.010521,6.225255e+12,6.841950e+12,3.118794e+13,...,3540.0,3750.0,3690.000000,3700.0,3528.385395,5.807300e+02,0.88301,0.136141,0.437086,4.080470
ACES,0.221650,1.257822,0.033536,3.810510,-3.494851e+12,36.083874,-0.056846,8.760908e+11,8.208312e+11,7.249255e+12,...,555.0,560.0,550.000000,555.0,346.005168,3.069000e+01,1.56185,0.091643,0.111956,24.386140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZATA,0.417536,5.011535,0.106133,2.475308,-1.082183e+11,-19.926090,1.451760,2.577930e+10,9.485564e+09,7.256928e+11,...,50.0,52.0,51.000000,52.0,60.256529,5.129029e+06,0.82979,0.007949,0.011269,15.120246
ZBRA,1.504979,0.466774,-0.154115,0.895548,-2.607505e+12,1.547369,-5.017143,1.538652e+10,-1.097531e+11,3.156274e+12,...,448.0,468.0,464.000000,466.0,501.850975,1.784000e+01,0.95336,-0.036948,-0.092553,51.892180
ZINC,2.202541,1.759970,-0.144765,0.728687,-5.399108e+11,6.082340,-2.284049,5.521285e+10,-1.288860e+11,2.475954e+12,...,50.0,50.0,1405.429907,50.0,30.618679,7.130000e+00,1.63299,-0.041569,-0.133125,25.831470


In [ ]:
type(scaled_data)

numpy.ndarray

In [ ]:
#aku ingin melihat range value supaya memastikan akan memakai treatment apa
range_values = df.agg(['min', 'max']).transpose()
print(range_values)

                                  min           max
total_asset_growth      -8.753910e+02  6.124804e+01
price_sales_ratio       -5.524135e+01  5.368061e+03
sales_growth            -1.230008e+00  9.667050e+01
quick_ratio              2.438673e-03  4.562098e+03
cost_of_goods_sold      -2.312910e+14  0.000000e+00
cash_flow_per_share     -5.087321e+04  5.231697e+04
net_income_growth       -1.101544e+03  4.779373e+02
operating_profit        -3.032963e+13  6.777461e+13
income_before_tax       -4.054456e+13  7.041540e+13
total_assets             6.453083e+07  1.992545e+15
total_equity            -2.414864e+13  3.033953e+14
total_liabilities        1.611488e+08  1.562244e+15
debt_to_equity_ratio    -8.753910e+02  6.124804e+01
cash_equivalent          0.000000e+00  6.404826e+13
total_sales             -1.016353e+11  3.013790e+14
operating_profit_margin -4.589907e+01  6.183977e+00
gross_profit_margin     -3.647534e+01  1.000000e+00
net_profit_margin       -1.202211e+03  1.548197e+02
net_income  

# Percobaan

In [ ]:
code = 'AALI'

In [ ]:
row_number = df.index.get_loc(code)
row_number

0

In [ ]:
user_profile = scaled_data[row_number] # Contoh profil pengguna

In [ ]:
df['similarity'] = cosine_similarity(scaled_data, [user_profile])

In [ ]:
df.head()

,total_asset_growth,price_sales_ratio,sales_growth,quick_ratio,cost_of_goods_sold,cash_flow_per_share,net_income_growth,operating_profit,income_before_tax,total_assets,...,highTradedPrice,bidPrice,offerPrice,book_value_per_share,eps,pbv,roa,roe,per,similarity
_id,,,,,,,,,,,,,,,,,,,,,
AALI,0.314978,0.707587,-0.102518,2.005423,-1.800647e+13,953.607381,-0.124157,2.360413e+12,2.429178e+12,2.924934e+13,...,7650.0,7650.0,7675.0,11556.791102,432.84,0.65260,0.059031,0.077624,10.22214,1.000000
ABBA,12.219118,3.491533,0.008439,1.051620,-9.834737e+10,-24.442054,-0.011553,-3.219792e+10,-2.743303e+10,3.888845e+11,...,107.0,104.0,105.0,7.474376,-9.49,-50.47942,-0.056624,-0.748516,-10.88109,-0.270498
ABDA,0.642063,5.343024,0.001388,80.622178,-2.570245e+11,-58.453396,-0.420802,1.186289e+11,1.187926e+11,2.472106e+12,...,0.0,6000.0,6450.0,2425.051349,237.60,2.66314,0.036869,0.060541,22.02523,0.391986
ABMM,2.210543,0.397122,0.559536,1.055253,-1.452951e+13,2861.056482,1.010521,6.225255e+12,6.841950e+12,3.118794e+13,...,3750.0,3690.0,3700.0,3528.385395,580.73,0.88301,0.136141,0.437086,4.08047,0.756979
ACES,0.221650,1.257822,0.033536,3.810510,-3.494851e+12,36.083874,-0.056846,8.760908e+11,8.208312e+11,7.249255e+12,...,560.0,550.0,555.0,346.005168,30.69,1.56185,0.091643,0.111956,24.38614,-0.129631


In [ ]:
recommendations = df.sort_values('similarity', ascending=False).head(10)

In [ ]:
print(recommendations)

      total_asset_growth  price_sales_ratio  sales_growth  quick_ratio  \
_id                                                                      
AALI            0.314978           0.707587     -0.102518     2.005423   
GGRM            0.530743           0.277774     -0.001590     0.267991   
INTP            0.313757           2.231968      0.105360     1.551463   
BRAM            0.307526           0.706116      0.422747     1.414672   
ITMG            0.353743           0.770880      0.930255     3.094971   
PRDA            0.155095           2.406444     -0.177439     6.167089   
UNIC            0.155024           0.589866      0.224255     3.208034   
TKIM            0.597901           1.220033      0.230665     0.816312   
INDR            0.869908           0.251051      0.167352     0.611497   
UNTR            0.569347           0.786872      0.555584     1.511506   

      cost_of_goods_sold  cash_flow_per_share  net_income_growth  \
_id                                        

In [ ]:
r